https://medium.com/thecyphy/train-cnn-model-with-pytorch-21dafb918f48

In [1]:
import torch
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
import glob
import shutil

Creat a subfolder with the correct label

In [ ]:
orig_images = '/data/kanferg/Images/Pex_project/SIngle_cell_images_training_set/01242024/input_sc_mix_10000/'
files = glob.glob(orig_images + "*png")
trainPEX_dir = '/data/kanferg/Images/Pex_project/SIngle_cell_images_training_set/01242024/pytorch_test/seg_train/PEX'
trainWT_dir = '/data/kanferg/Images/Pex_project/SIngle_cell_images_training_set/01242024/pytorch_test/seg_train/WT'
testPEX_dir = '/data/kanferg/Images/Pex_project/SIngle_cell_images_training_set/01242024/pytorch_test/seg_test/PEX'
testWT_dir = '/data/kanferg/Images/Pex_project/SIngle_cell_images_training_set/01242024/pytorch_test/seg_test/WT'

In [79]:
filePEX = []
fileWT = []
for file in files:
    if 'WT' in os.path.basename(file):
        fileWT.append(file)
    else:
        filePEX.append(file)
fileWTCount = int(len(fileWT)*0.93)
filePEXCount = int(len(filePEX)*0.93)
print(fileWTCount),print(filePEXCount)

7903
7773


(None, None)

In [81]:
# for i in range(7500):
#     shutil.copy(fileWT[i], trainWT_dir)
#     shutil.copy(filePEX[i], trainPEX_dir)
#     print('.' * i, end = '\r')
# for i in range(7501,7773):
#     shutil.copy(fileWT[i], testWT_dir)
#     shutil.copy(filePEX[i], testPEX_dir)
#     print('.' * i, end = '\r')

In [82]:
data_dir = '/data/kanferg/Images/Pex_project/SIngle_cell_images_training_set/01242024/pytorch_test/seg_train/'
test_data_dir = '/data/kanferg/Images/Pex_project/SIngle_cell_images_training_set/01242024/pytorch_test/seg_test/'

In [83]:
dataset = ImageFolder(data_dir,transform = transforms.Compose([
    transforms.Resize((150,150)),transforms.ToTensor()
]))
test_dataset = ImageFolder(test_data_dir,transforms.Compose([
    transforms.Resize((150,150)),transforms.ToTensor()
]))

In [84]:
img, label = dataset[0]
print(img.shape,label)


torch.Size([3, 150, 150]) 0


In [85]:
print("Follwing classes are there : \n",dataset.classes)

Follwing classes are there : 
 ['PEX', 'WT']


In [89]:
import torch.nn as nn
import torch.nn.functional as F

class ImageClassificationBase(nn.Module):
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

In [86]:
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split

batch_size = 30
val_size = 3000
train_size = len(dataset) - val_size 

train_data,val_data = random_split(dataset,[train_size,val_size])
print(f"Length of Train Data : {len(train_data)}")
print(f"Length of Validation Data : {len(val_data)}")

#output
#Length of Train Data : 15000*0.8
#Length of Validation Data : 15000*0.2

#load the train and validation into batches.
train_dl = DataLoader(train_data, batch_size, shuffle = True, num_workers = 4, pin_memory = True)
val_dl = DataLoader(val_data, batch_size*2, num_workers = 4, pin_memory = True)

Length of Train Data : 12000
Length of Validation Data : 3000


In [ ]:
size_after_convs = 150
for _ in range(4):  # 4 sets of Conv2D + MaxPooling2D
    size_after_convs = (size_after_convs - 2) // 2  # Kernel size 3, stride 1 for conv and 2 for pool
flat_size = size_after_convs * size_after_convs * 128  # The last number of channels after convolutions
flat_size

In [90]:
class NaturalSceneClassification(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            
            nn.Conv2d(3, 16, kernel_size = (3,3), padding = 'valid'),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2)), # Output size: [16, 74, 74]
        
            nn.Conv2d(16, 64, kernel_size = (3,3), stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2)), # Output size: [64, 36, 36]
            
            nn.Conv2d(64, 128, kernel_size = (3,3), stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2)), # Output size: [128, 17, 17]
            
            nn.Conv2d(128, 128, kernel_size = (3,3), stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2)), # Output size: [128, 7, 7]
            
            nn.Flatten(),
            nn.Linear(128 * 7 * 7, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(512, 1),
            nn.Sigmoid()
        )
    
    def forward(self, xb):
        return self.network(xb)

In [91]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

  
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

  
def fit(epochs, lr, model, train_loader, val_loader, opt_func = torch.optim.SGD):
    
    history = []
    optimizer = opt_func(model.parameters(),lr)
    for epoch in range(epochs):
        
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    
    return history

In [93]:
model = NaturalSceneClassification()

In [100]:
num_epochs = 30
opt_func = torch.optim.Adam
lr = 0.001
#fitting the model on training data and record the result after each epoch
history = fit(num_epochs, lr, model, train_dl, val_dl, opt_func)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (30x10368 and 6272x512)

In [ ]:
%debug

> /usr/local/Anaconda/envs/py3.9/lib/python3.9/site-packages/torch/nn/modules/linear.py(103)forward()
    101 
    102     def forward(self, input: Tensor) -> Tensor:
--> 103         return F.linear(input, self.weight, self.bias)
    104 
    105     def extra_repr(self) -> str:

ipdb> whos
*** NameError: name 'whos' is not defined


In [97]:
model.training_step(batch)

NameError: name 'batch' is not defined

In [99]:
%debug

> /tmp/ipykernel_1927558/2448957765.py(1)<module>()
----> 1 model.training_step(batch)

ipdb> locals()


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



ipdb> locals()
ipdb> args
ipdb> args()
ipdb> locals
<built-in function locals>
ipdb> locals()


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



ipdb> images
*** NameError: name 'images' is not defined
ipdb> whos
*** NameError: name 'whos' is not defined
ipdb> s
